In [2]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [3]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=9, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

In [4]:
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

# n_pop = 10
# n_gen = 10
# n_sats = 5
# opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-8,-1))

In [ ]:
stk_object.dt = 30
stk_object.Interpolate=True

# Run optimization if read=True the 1st generation takes the input file as one of the initial population
hof,percent,std,time = opt.run(read=True)

print("Hall of Fame:")
for entry in hof:
    print(entry,"->",entry.fitness)

# Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

best_stats = opt.cost_function(hof[0])

In [6]:
stk_object.dt = 2.5
stk_object.Generate_Holding_Data()

- Computing_AER Data/Default [==========] 180/180 [100%] in 2:35.6 (1.15/s)              ] ▁▃▅ 7/180 [4%] in 5s (~2:03, 1.4/s) 
- Computing_Sat Angles Data [==========] 180/180 [100%] in 1:01.3 (2.94/s)      


0

In [7]:
stk_object.Plan(slew_rate=1)

- Holding_Data [==========] 4860/4860 [100%] in 3.3s (1570.44/s)                [==========] ▄▆█ 4672/4860 [96%] in 3s (~0s, 1871.2/s) 


0

In [37]:
target_times = []
with alive_bar(324*len(stk_object.targets),force_tty=True,bar='classic',title=f'- Timing Planned Data',length=10) as bar:
    for tar_num in range(len(stk_object.targets)):
        times = []
        for bin_num in range(324):
            tar_window = stk_object.Holding_Data['Target'].values==tar_num
            bin_window = stk_object.Holding_Data['Bin Number'].values==bin_num
            times.append(min(stk_object.Holding_Data['Time'].values[tar_window&bin_window]))
            bar()
        target_times.append(max(times))

data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = np.array(target_times)/86400
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

- Timing Planned Data [==========] 4860/4860 [100%] in 42.1s (115.58/s)         [==========] ▃▁▃ 4673/4860 [96%] in 40s (~2s, 115.4/s) 
    Unplanned (%)  Planned (%)  Unplanned (Time)  Planned (Time)
0           100.0        100.0          5.346834        5.346834
1           100.0        100.0          3.287164        3.287164
2           100.0        100.0          8.213762        8.213762
3           100.0        100.0          2.553791        2.553791
4           100.0        100.0          3.421690        3.421690
5           100.0        100.0          5.894994        5.894994
6           100.0        100.0          8.441273        8.441273
7           100.0        100.0          1.012188        1.012188
8           100.0        100.0          4.225330        4.225330
9           100.0        100.0          7.883600        7.883600
10          100.0        100.0          5.345440        5.345440
11          100.0        100.0          6.565104        6.565104
12          100.0  

In [38]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()